In [ ]:
# For general text generation
from llm.general import OpenAIGeneral

general_client = OpenAIGeneral(api_key="your-openai-api-key")
response = general_client.generate("Write a short story about a robot")

# For structured output
from llm.structured import OpenAIStructured

structured_client = OpenAIStructured(api_key="your-openai-api-key")
json_response = structured_client.generate(
    "Extract the following entities from this text: John Smith visited Paris on June 15, 2024"
)